# Imports

In [1]:
import requests
import json
import os
import numpy as np
import pandas as pd

# Definitions

In [2]:
cwd = os.getcwd()
leagueID = "1041760"
live_gw_template = "https://fantasy.premierleague.com/api/event/{gw}/live/"
bootstrap_static_url = "https://fantasy.premierleague.com/api/bootstrap-static/"

# Functions

In [3]:
def get_requests_response(url_template, **kwargs):
    response = requests.get(url_template.format(**kwargs))
    response_json = response.json()
    return response_json

# Testing

## Get players data per GW

In [4]:
all_players_all_gw_list = []
for gw in range(1, 26):
    """if gw != 1:
    continue"""
    response_json = get_requests_response(url_template=live_gw_template, gw=gw)
    all_players_dict_list = []
    for result_dict in response_json["elements"]:
        player_id = result_dict["id"]
        stats_dict = result_dict["stats"]
        stats_dict["player_id"] = player_id
        all_players_dict_list.append(stats_dict)
    all_players_df = pd.DataFrame(all_players_dict_list)
    all_players_df["gw"] = gw
    all_players_all_gw_list.append(all_players_df)
all_players_df = pd.concat(all_players_all_gw_list).reset_index(drop=True)
all_players_df.columns

Index(['minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_conceded', 'total_points',
       'in_dreamteam', 'player_id', 'gw'],
      dtype='object')

## Open player data and make ID map

In [5]:
bootstrap_static_response = get_requests_response(bootstrap_static_url, kwars={})
players_df = pd.DataFrame(bootstrap_static_response["elements"]).loc[
    :, ["id", "web_name"]
]
id_name_dict = pd.Series(
    players_df["web_name"].values, index=players_df["id"]
).to_dict()

In [6]:
all_players_df["player_name"] = all_players_df["player_id"].map(id_name_dict)
all_players_df

,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,...,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,total_points,in_dreamteam,player_id,gw,player_name
0,0,0,0,0,0,0,0,0,0,0,...,0,0.00,0.00,0.00,0.00,0,False,1,1,Balogun
1,0,0,0,0,0,0,0,0,0,0,...,0,0.00,0.00,0.00,0.00,0,False,2,1,Cédric
2,0,0,0,0,0,0,0,0,0,0,...,0,0.00,0.00,0.00,0.00,0,False,3,1,M.Elneny
3,0,0,0,0,0,0,0,0,0,0,...,0,0.00,0.00,0.00,0.00,0,False,4,1,Fábio Vieira
4,4,0,0,0,0,0,0,0,0,0,...,0,0.00,0.00,0.00,0.02,1,False,5,1,Gabriel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18655,0,0,0,0,0,0,0,0,0,0,...,0,0.00,0.00,0.00,0.00,0,False,820,25,Harrison
18656,0,0,0,0,0,0,0,0,0,0,...,0,0.00,0.00,0.00,0.00,0,False,821,25,Umeh-Chibueze
18657,0,0,0,0,0,0,0,0,0,0,...,0,0.00,0.00,0.00,0.00,0,False,822,25,Blacker
18658,0,0,0,0,0,0,0,0,0,0,...,0,0.00,0.00,0.00,0.00,0,False,823,25,Arblaster


## Reduce columns

In [7]:
player_gw_points_df = all_players_df.loc[:, ["player_name", "gw", "total_points"]]
player_gw_points_df

,player_name,gw,total_points
0,Balogun,1,0
1,Cédric,1,0
2,M.Elneny,1,0
3,Fábio Vieira,1,0
4,Gabriel,1,1
...,...,...,...
18655,Harrison,25,0
18656,Umeh-Chibueze,25,0
18657,Blacker,25,0
18658,Arblaster,25,0
